# Explora aquí

Se recomienda utilizar este cuaderno con fines de exploración.

In [1]:
import os
from bs4 import BeautifulSoup
import requests
import time
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sqlite3

print("All previous libraries are imported correctly!")

All previous libraries are imported correctly!


### DOWNLOAD HTML

In [2]:
# We specify the url we want to scrape:
url = 'https://ycharts.com/companies/TSLA/revenues'

# Headers para simular una solicitud desde un navegador real [NOS SALÍA ERROR 403]
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36',
    'Accept-Language': 'en-US,en;q=0.9',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1'
}

try:
    # Hacemos la solicitud HTTP GET con los headers
    response = requests.get(url, headers=headers)

    # Verificamos el código de estado de la respuesta
    if response.status_code == 200:
        print("Acceso exitoso!")
    else:
        print(f"No hemos podido acceder, nos sale el error: {response.status_code}")
except Exception as e:
    print(f"Ocurrió un error: {e}")

Acceso exitoso!


In [3]:
response = requests.get(url, headers=headers)
content = response.content

soup = BeautifulSoup(content, 'html.parser')

soup


<!DOCTYPE html>

<html lang="en" ng-jq="">
<head>
<title>
        Tesla Revenue (Quarterly) Analysis | YCharts
        </title>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<!-- This code allows us to correctly display the page on mobile devices -->
<script defer="">
    var viewport = document.querySelector('meta[name=viewport]');
    if (viewport) {
        var minWidth = 700;
        if (screen.width < minWidth) {
            document.head.removeChild(viewport);
            var newViewport = document.createElement('meta');
            newViewport.setAttribute('name', 'viewport');
            newViewport.setAttribute('content', 'width=' + minWidth);
            document.head.appendChild(newViewport);
        }
    }
</script>
<script>
    if (navigator.userAgent.indexOf('Trident') !== -1 || navigator.userAgent.indexOf('MSIE') !== -1) {
        window.onload=function() {
            var ie_banner = document.createElement('div');
            ie_banner.innerT

In [4]:
# We now find the tables.
tables = soup.find_all('table', class_='table')
# We check how many we found.
print(f'We found {len(tables)} tables.')
print(f'--------------------')
# We print the name of the four tables.
table_names = soup.find_all('h3', class_='panel-title')
for name in table_names:
    print(name.get_text().strip())

# We find inconsistency in the number of tables and titles. Therefore we understand the titles might not all have a table to itself.



We found 4 tables.
--------------------
Revenue (Quarterly) Chart
Historical Revenue (Quarterly) Data
Revenues Definition
Revenue (Quarterly) Range, Past 5 Years
Revenue (Quarterly) Benchmarks
Revenue (Quarterly) Related Metrics


In [5]:
# From the list of titles, we find that Historical Revenue (Quarterly) Data is the second table (position 1 index).
revenue_table = tables[1]

# We create a variable for the rows in the revenue table.
rows = revenue_table.find_all('tr')

# We create a variable for the headers of the revenue table.
headers = [header.get_text(strip=True) for header in rows[0].find_all('th')]

# We create a list to store the data. 
data = []

# We iterate through the rows of the table to fill the data list.
for row in rows[1:]:
    columns = row.find_all('td')
    data.append([column.get_text(strip=True) for column in columns])

# We create the DataFrame from the data list.
df = pd.DataFrame(data, columns=headers)


print(df)

                  Date    Value
0       March 31, 2018   3.409B
1    December 31, 2017   3.288B
2   September 30, 2017   2.985B
3        June 30, 2017   2.790B
4       March 31, 2017   2.696B
5    December 31, 2016   2.285B
6   September 30, 2016   2.298B
7        June 30, 2016   1.270B
8       March 31, 2016   1.147B
9    December 31, 2015   1.214B
10  September 30, 2015  936.79M
11       June 30, 2015  954.98M
12      March 31, 2015  939.88M
13   December 31, 2014  956.66M
14  September 30, 2014  851.80M
15       June 30, 2014  769.35M
16      March 31, 2014  620.54M
17   December 31, 2013  615.22M
18  September 30, 2013  431.35M
19       June 30, 2013  405.14M
20      March 31, 2013  561.79M
21   December 31, 2012  306.33M
22  September 30, 2012   50.10M
23       June 30, 2012   26.65M
24      March 31, 2012   30.17M


To be able to analise the data, we will clean the column value, and the column date.

In [6]:
# Function to convert the values to the real number.
def convert_to_number(value):
    if 'B' in value:
        return float(value.replace('B', '')) * 1000000000
    elif 'M' in value:
        return float(value.replace('M', '')) * 1000000
    else:
        return float(value)
    
# We run the function over the value column 
df['Value'] = df['Value'].apply(convert_to_number)

# We check the function edited the information correctly.
df['Value'] = df['Value'].apply(lambda x: f"{x:,.0f}".replace(",", ".")) # Set pandas to display floats without scientific notation and replace , for .

In [7]:
df.head()

,Date,Value
0,"March 31, 2018",3.409.000.000
1,"December 31, 2017",3.288.000.000
2,"September 30, 2017",2.985.000.000
3,"June 30, 2017",2.790.000.000
4,"March 31, 2017",2.696.000.000


In [8]:
df.tail()

,Date,Value
20,"March 31, 2013",561.790.000
21,"December 31, 2012",306.330.000
22,"September 30, 2012",50.100.000
23,"June 30, 2012",26.650.000
24,"March 31, 2012",30.170.000


In [9]:
# We left the dots in the value column to visually confirm it is transforming correctly. We proceed to clear the "." from the value column.
df["Value"] = df["Value"].str.replace(".", "", regex=False)

df.head()

,Date,Value
0,"March 31, 2018",3409000000
1,"December 31, 2017",3288000000
2,"September 30, 2017",2985000000
3,"June 30, 2017",2790000000
4,"March 31, 2017",2696000000


We will now generate with SQLite the database in the src folder.

In [10]:
db_path = os.path.join(os.getcwd(), 'database.db')

# Print the db_path to verify it's correct
print(f"Database path: {db_path}")

# Connect to the SQLite database (this will create the file if it doesn't exist)
try:
    conn = sqlite3.connect(db_path)
    print(f"Created or connected to the database file: {db_path}")
    conn.close()
except sqlite3.OperationalError as e:
    print(f"OperationalError: {e}")

Database path: d:\01A-TRABAJO\PYTHON\DATASCIENCE\WEBSCRAPING\web-scraping-exercise\src\database.db
Created or connected to the database file: d:\01A-TRABAJO\PYTHON\DATASCIENCE\WEBSCRAPING\web-scraping-exercise\src\database.db


In [11]:
# We connect to the database and create a table named historic_revenue to store the data.
conn = sqlite3.connect(db_path)
cur = conn.cursor()

cur.execute("CREATE TABLE IF NOT EXISTS historic_revenue (Date, Value)")


In [12]:
# We create a tuple with the data in the dataframe, we do not include the index column.
df_tuples = [tuple(x) for x in df.to_records(index=False)]

# We check the tuples did get in correctly.
print(df_tuples)
df_tuples[:5]


[('March 31, 2018', '3409000000'), ('December 31, 2017', '3288000000'), ('September 30, 2017', '2985000000'), ('June 30, 2017', '2790000000'), ('March 31, 2017', '2696000000'), ('December 31, 2016', '2285000000'), ('September 30, 2016', '2298000000'), ('June 30, 2016', '1270000000'), ('March 31, 2016', '1147000000'), ('December 31, 2015', '1214000000'), ('September 30, 2015', '936790000'), ('June 30, 2015', '954980000'), ('March 31, 2015', '939880000'), ('December 31, 2014', '956660000'), ('September 30, 2014', '851800000'), ('June 30, 2014', '769350000'), ('March 31, 2014', '620540000'), ('December 31, 2013', '615220000'), ('September 30, 2013', '431350000'), ('June 30, 2013', '405140000'), ('March 31, 2013', '561790000'), ('December 31, 2012', '306330000'), ('September 30, 2012', '50100000'), ('June 30, 2012', '26650000'), ('March 31, 2012', '30170000')]


[('March 31, 2018', '3409000000'),
 ('December 31, 2017', '3288000000'),
 ('September 30, 2017', '2985000000'),
 ('June 30, 2017', '2790000000'),
 ('March 31, 2017', '2696000000')]

In [13]:
# We insert the data into the table.
cur.executemany("INSERT INTO historic_revenue (Date, Value) VALUES (?,?)", df_tuples)
conn.commit()
conn.close()

PENDIENTE - LIMPIAR LAS FECHAS PARA QUE SEAN dd/mm/aaaa / HACER VISUALES 

último punto Visualiza los datos

¿Qué tipos de visualizaciones podemos realizar? Propón al menos 3 y muéstralos.